## Introduction

[![Binder](https://notebooks.gesis.org/binder/badge_logo.svg)](https://notebooks.gesis.org/binder/v2/gh/imperial-genomics-facility/scanpy-notebook-image/v2?urlpath=lab)
This notebook uses data set ([GSE92332](https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE92332)) of different regions of the mouse intestinal epithelium from (Haber et al. 2018) and uses codes and documentation from [Best practices in single-cell RNA-seq analysis: a tutorial](https://github.com/theislab/single-cell-tutorial) for data analsis using a variety of tools.

##  Loading the libraries

We need to load all the required libraries to environment before we can run any of the analysis steps. Also, we are checking the version information for most of the major packages used for analysis.

In [ ]:
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
from gprofiler import GProfiler

import rpy2.rinterface_lib.callbacks
import logging

from rpy2.robjects import pandas2ri
import anndata2ri

In [ ]:
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()

In [ ]:
%%R
# Load libraries from correct lib Paths for my environment - ignore this!
#.libPaths(.libPaths()[c(3,2,1)])

# Load all the R libraries we will be using in the notebook
library(scran)
library(RColorBrewer)
library(slingshot)
library(monocle)
library(gam)
library(clusterExperiment)
library(ggplot2)
library(plyr)
library(MAST)

## Fetch raw data from NCBI GEO

The following steps are only required for downloading test data from NCBI GEO.

In [ ]:
!rm -rf /tmp/GSE92332_RAW;mkdir /tmp/GSE92332_RAW
!cd /tmp/GSE92332_RAW;\
wget -q -O /tmp/GSE92332_RAW/GSE92332_RAW.tar  \
  http://ftp.ncbi.nlm.nih.gov/geo/series/GSE92nnn/GSE92332/suppl/GSE92332_RAW.tar
!cd /tmp/GSE92332_RAW;tar -xf GSE92332_RAW.tar

In [ ]:
# TODO
#!wget -q -O /tmp/Macosko_cell_cycle_genes.txt \
#https://raw.githubusercontent.com/theislab/single-cell-tutorial/master/Macosko_cell_cycle_genes.txt

## Reading in the data

Scanpy expects the data to be stored in the format cells x genes, so we need to transpose the data matrix.

In [ ]:
# Set up data loading, we are using only single dataset for each type of cell type due to memory limitation

#Data files
sample_strings = ['Duo_M1', 'Jej_M1',  'Il_M1']
sample_id_strings = ['3', '5','7']
file_base = '/tmp/GSE92332_RAW/GSM283657'
exp_string = '_Regional_'
data_file_end = '_matrix.mtx.gz'
barcode_file_end = '_barcodes.tsv.gz'
gene_file_end = '_genes.tsv.gz'
cc_genes_file = '/tmp/Macosko_cell_cycle_genes.txt'

In [ ]:
# First data set load & annotation
#Parse Filenames
sample = sample_strings.pop(0)
sample_id = sample_id_strings.pop(0)
data_file = file_base+sample_id+exp_string+sample+data_file_end
barcode_file = file_base+sample_id+exp_string+sample+barcode_file_end
gene_file = file_base+sample_id+exp_string+sample+gene_file_end

#Load data
adata = sc.read(data_file, cache=True)
adata = adata.transpose()
adata.X = adata.X.toarray()

barcodes = pd.read_csv(barcode_file, header=None, sep='\t')
genes = pd.read_csv(gene_file, header=None, sep='\t')

#Annotate data
barcodes.rename(columns={0:'barcode'}, inplace=True)
barcodes.set_index('barcode', inplace=True)
adata.obs = barcodes
adata.obs['sample'] = [sample]*adata.n_obs
adata.obs['region'] = [sample.split("_")[0]]*adata.n_obs
adata.obs['donor'] = [sample.split("_")[1]]*adata.n_obs

genes.rename(columns={0:'gene_id', 1:'gene_symbol'}, inplace=True)
genes.set_index('gene_symbol', inplace=True)
adata.var = genes

In [ ]:
# Loop to load rest of data sets
for i in range(len(sample_strings)):
    #Parse Filenames
    sample = sample_strings[i]
    sample_id = sample_id_strings[i]
    data_file = file_base+sample_id+exp_string+sample+data_file_end
    barcode_file = file_base+sample_id+exp_string+sample+barcode_file_end
    gene_file = file_base+sample_id+exp_string+sample+gene_file_end
    
    #Load data
    adata_tmp = sc.read(data_file, cache=True)
    adata_tmp = adata_tmp.transpose()
    adata_tmp.X = adata_tmp.X.toarray()

    barcodes_tmp = pd.read_csv(barcode_file, header=None, sep='\t')
    genes_tmp = pd.read_csv(gene_file, header=None, sep='\t')
    
    #Annotate data
    barcodes_tmp.rename(columns={0:'barcode'}, inplace=True)
    barcodes_tmp.set_index('barcode', inplace=True)
    adata_tmp.obs = barcodes_tmp
    adata_tmp.obs['sample'] = [sample]*adata_tmp.n_obs
    adata_tmp.obs['region'] = [sample.split("_")[0]]*adata_tmp.n_obs
    adata_tmp.obs['donor'] = [sample.split("_")[1]]*adata_tmp.n_obs
    
    genes_tmp.rename(columns={0:'gene_id', 1:'gene_symbol'}, inplace=True)
    genes_tmp.set_index('gene_symbol', inplace=True)
    adata_tmp.var = genes_tmp
    adata_tmp.var_names_make_unique()

    # Concatenate to main adata object
    adata = adata.concatenate(adata_tmp, batch_key='sample_id')
    #adata.var['gene_id'] = adata.var['gene_id-1']
    #adata.var.drop(columns = ['gene_id-1', 'gene_id-0'], inplace=True)
    adata.obs.drop(columns=['sample_id'], inplace=True)
    adata.obs_names = [c.split("-")[0] for c in adata.obs_names]
    adata.obs_names_make_unique(join='_')

In [ ]:
#Assign variable names and gene id columns
adata.var_names = [g.split("_")[1] for g in adata.var_names]
adata.var['gene_id'] = [g.split("_")[1] for g in adata.var['gene_id']]

In [ ]:
# Annotate the data sets
print(adata.obs['region'].value_counts())
print('')
print(adata.obs['donor'].value_counts())
print('')
print(adata.obs['sample'].value_counts())

In [ ]:
# Checking the total size of the data set
adata.shape

Scanpy stores the count data is an annotated data matrix (observations e.g. cell barcodes × variables e.g gene names) called AnnData together with annotations of observations __(obs)__, variables __(var)__ and unstructured annotations __(uns)__

## Pre-processing and visualization

### Quality control

A high fraction of mitochondrial reads being picked up can indicate cell stress, as there is a low proportion of nuclear mRNA in the cell. It should be noted that high mitochondrial RNA fractions can also be biological signals indicating elevated respiration.

Typical quality measures for assessing the quality of a cell includes the following components

* Number of molecule counts (UMIs or n_counts )
* Number of expressed genes (n_genes)
* Fraction of counts that are mitochondrial (percent_mito)

We are calculating the above mentioned details using the following codes.

Cell barcodes with high count depth, few detected genes and high fraction of mitochondrial counts may indicate cells whose cytoplasmic mRNA has leaked out due to a broken membrane and only the mRNA located in the mitochondria has survived.

Cells with high UMI counts and detected genes may represent doublets (it requires further checking).

In [ ]:
# Quality control - calculate QC covariates
adata.obs['n_counts'] = adata.X.sum(1)
adata.obs['log_counts'] = np.log(adata.obs['n_counts'])
adata.obs['n_genes'] = (adata.X > 0).sum(1)

mt_gene_mask = [gene.startswith('mt-') for gene in adata.var_names]
adata.obs['mt_frac'] = adata.X[:, mt_gene_mask].sum(1)/adata.obs['n_counts']

In [ ]:
# Quality control - plot QC metrics
#Sample quality plots
t1 = sc.pl.violin(adata, 'n_counts', groupby='sample', size=2, log=True, cut=0)
t2 = sc.pl.violin(adata, 'mt_frac', groupby='sample')

In [ ]:
#Data quality summary plots
p1 = sc.pl.scatter(adata, 'n_counts', 'n_genes', color='mt_frac')
p2 = sc.pl.scatter(adata[adata.obs['n_counts']<10000], 'n_counts', 'n_genes', color='mt_frac')

In [ ]:
#Thresholding decision: counts
p3 = sb.distplot(adata.obs['n_counts'], kde=False)
plt.show()

p4 = sb.distplot(adata.obs['n_counts'][adata.obs['n_counts']<4000], kde=False, bins=60)
plt.show()

p5 = sb.distplot(adata.obs['n_counts'][adata.obs['n_counts']>10000], kde=False, bins=60)
plt.show()

In [ ]:
#Thresholding decision: genes
p6 = sb.distplot(adata.obs['n_genes'], kde=False, bins=60)
plt.show()

p7 = sb.distplot(adata.obs['n_genes'][adata.obs['n_genes']<1000], kde=False, bins=60)
plt.show()

In [ ]:
# Filter cells according to identified QC thresholds:
print('Total number of cells: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, min_counts = 1500)
print('Number of cells after min count filter: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, max_counts = 40000)
print('Number of cells after max count filter: {:d}'.format(adata.n_obs))

adata = adata[adata.obs['mt_frac'] < 0.2]
print('Number of cells after MT filter: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, min_genes = 700)
print('Number of cells after gene filter: {:d}'.format(adata.n_obs))

In [ ]:
#Filter genes:
print('Total number of genes: {:d}'.format(adata.n_vars))

# Min 20 cells - filters out 0 count genes
sc.pp.filter_genes(adata, min_cells=20)
print('Number of genes after cell filter: {:d}'.format(adata.n_vars))

### Normalization

In [ ]:
#Perform a clustering for scran normalization in clusters
adata_pp = adata.copy()
sc.pp.normalize_per_cell(adata_pp, counts_per_cell_after=1e6)
sc.pp.log1p(adata_pp)
sc.pp.pca(adata_pp, n_comps=15)
sc.pp.neighbors(adata_pp)
sc.tl.louvain(adata_pp, key_added='groups', resolution=0.5)

In [ ]:
#Preprocess variables for scran normalization
input_groups = adata_pp.obs['groups']
data_mat = adata.X.T

In [ ]:
%%R -i data_mat -i input_groups -o size_factors

size_factors = computeSumFactors(data_mat, clusters=input_groups, min.mean=0.1)

In [ ]:
#Delete adata_pp
del adata_pp
del data_mat
del input_groups

In [ ]:
# Visualize the estimated size factors
adata.obs['size_factors'] = size_factors

sc.pl.scatter(adata, 'size_factors', 'n_counts')
sc.pl.scatter(adata, 'size_factors', 'n_genes')

sb.distplot(size_factors, bins=50, kde=False)
plt.show()

In [ ]:
#Keep the count data in a counts layer
adata.layers["counts"] = adata.X.copy()

In [ ]:
#Normalize adata 
adata.X /= adata.obs['size_factors'].values[:,None]
sc.pp.log1p(adata)

In [ ]:
import scipy
adata.X = scipy.sparse.csr_matrix(adata.X)

In [ ]:
# Store the full data set in 'raw' as log-normalised data for statistical testing
adata.raw = adata

### Batch Correction

In [ ]:
# ComBat batch correction
sc.pp.combat(adata, key='sample')

### Highly Variable Genes

In [ ]:
sc.pp.highly_variable_genes(adata, flavor='cell_ranger', n_top_genes=4000)
print('\n','Number of highly variable genes: {:d}'.format(np.sum(adata.var['highly_variable'])))

In [ ]:
sc.pl.highly_variable_genes(adata)

### Visualization

In [ ]:
# Calculate the visualizations
sc.pp.pca(adata, n_comps=40, use_highly_variable=True, svd_solver='arpack')
sc.pp.neighbors(adata)

sc.tl.tsne(adata, n_jobs=2) #Note n_jobs works for MulticoreTSNE, but not regular implementation)
sc.tl.umap(adata)
sc.tl.diffmap(adata)
sc.tl.draw_graph(adata)

In [ ]:
sc.pl.pca_scatter(adata, color='n_counts')
sc.pl.tsne(adata, color='n_counts')
sc.pl.umap(adata, color='n_counts')
sc.pl.diffmap(adata, color='n_counts', components=['1,2','1,3'])
sc.pl.draw_graph(adata, color='n_counts')

In [ ]:
sc.tl.leiden(adata)

In [ ]:
from copy import deepcopy
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
leiden_series = deepcopy(adata.obs['leiden'])
cell_clusters = list(leiden_series.value_counts().to_dict().keys())
colors = sc.pl.palettes.default_102[0:len(cell_clusters) ]
dict_map = dict(zip(cell_clusters,colors))
color_map = leiden_series.map(dict_map).values
labels = list(adata.obs.index)

sc.tl.umap(
  adata,
  n_components=3)
hovertext = \
  ['cluster: {0}, barcode: {1}'.\
     format(
       grp,labels[index])
         for index,grp in enumerate(leiden_series.values)]
## plotting 3D UMAP as html file
plot(
  [go.Scatter3d(
     x=adata.obsm['X_umap'][:, 0],
     y=adata.obsm['X_umap'][:, 1],
     z=adata.obsm['X_umap'][:, 2], 
     mode='markers',
     marker=dict(color=color_map,
                 size=5),
     opacity=0.6,
     text=labels,
     hovertext=hovertext,
  )],
  filename='UMAP-3D-plot.html')

In [ ]:
sc.tl.umap(adata)

In [ ]:
plt.rcParams['figure.figsize']=(11,9)
sc.pl.umap(adata, color=['leiden'],use_raw=False,palette=sc.pl.palettes.default_102)

In [ ]:
sc.pl.umap(adata, color=['sample'],use_raw=False)

### Cell cycle scoring

In [ ]:
#Score cell cycle and visualize the effect:
cc_genes = pd.read_table(cc_genes_file, delimiter='\t')
s_genes = cc_genes['S'].dropna()
g2m_genes = cc_genes['G2.M'].dropna()

s_genes_mm = [gene.lower().capitalize() for gene in s_genes]
g2m_genes_mm = [gene.lower().capitalize() for gene in g2m_genes]

s_genes_mm_ens = adata.var_names[np.in1d(adata.var_names, s_genes_mm)]
g2m_genes_mm_ens = adata.var_names[np.in1d(adata.var_names, g2m_genes_mm)]

sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes_mm_ens, g2m_genes=g2m_genes_mm_ens)

In [ ]:
sc.pl.umap(adata, color='S_score', use_raw=False)
sc.pl.umap(adata, color='G2M_score', use_raw=False)
sc.pl.umap(adata, color='phase', use_raw=False)

## Downstream analysis

### Clustering

In [ ]:
# Perform clustering - using highly variable genes
sc.tl.louvain(adata, key_added='louvain_r1')
sc.tl.louvain(adata, resolution=0.5, key_added='louvain_r0.5', random_state=10)

In [ ]:
adata.obs['louvain_r1'].value_counts()

In [ ]:
adata.obs['louvain_r0.5'].value_counts()

In [ ]:
adata.obs['leiden'].value_counts()

In [ ]:
sc.tl.louvain(adata, resolution=0.6, key_added='louvain_r0.6', random_state=10)

In [ ]:
adata.obs['louvain_r0.6'].value_counts()

In [ ]:
sc.tl.leiden(adata, resolution=0.6, key_added='leiden_r0.6', random_state=10)

In [ ]:
adata.obs['leiden_r0.6'].value_counts()

In [ ]:
#Visualize the clustering and how this is reflected by different technical covariates
sc.pl.umap(adata, color='louvain_r1', palette=sc.pl.palettes.default_102)
sc.pl.umap(adata, color='louvain_r0.5', palette=sc.pl.palettes.default_102)
sc.pl.umap(adata, color='louvain_r0.6', palette=sc.pl.palettes.default_102)
sc.pl.umap(adata, color='leiden', palette=sc.pl.palettes.default_102)
sc.pl.umap(adata, color='leiden_r0.6', palette=sc.pl.palettes.default_102)
sc.pl.umap(adata, color=['region', 'n_counts'])
sc.pl.umap(adata, color=['log_counts', 'mt_frac'])

### Marker genes & cluster annotation

In [ ]:
#Calculate marker genes
sc.tl.rank_genes_groups(adata, groupby='leiden_r0.6', key_added='rank_genes_leiden_r0.6')

In [ ]:
#Plot marker genes
sc.pl.rank_genes_groups(adata, key='rank_genes_leiden_r0.6',ncols=2)

In [ ]:
#Known marker genes:
marker_genes = dict()
marker_genes['Stem'] = ['Lgr5', 'Ascl2', 'Slc12a2', 'Axin2', 'Olfm4', 'Gkn3']
marker_genes['Enterocyte (Proximal)'] = ['Gsta1','Rbp2','Adh6a','Apoa4','Reg3a','Creb3l3','Cyp3a13','Cyp2d26','Ms4a10','Ace','Aldh1a1','Rdh7','H2-Q2', 'Hsd17b6','Gstm3','Gda','Apoc3','Gpd1','Fabp1','Slc5a1','Mme','Cox7a1','Gsta4','Lct','Khk','Mttp','Xdh','Sult1b1', 'Treh','Lpgat1','Dhrs1','Cyp2c66','Ephx2','Cyp2c65','Cyp3a25','Slc2a2','Ugdh','Gstm6','Retsat','Ppap2a','Acsl5', 'Cyb5r3','Cyb5b','Ckmt1','Aldob','Ckb','Scp2','Prap1']
marker_genes['Enterocyte (Distal)'] = ['Tmigd1','Fabp6','Slc51b','Slc51a','Mep1a','Fam151a','Naaladl1','Slc34a2','Plb1','Nudt4','Dpep1','Pmp22','Xpnpep2','Muc3','Neu1','Clec2h','Phgr1','2200002D01Rik','Prss30','Cubn','Plec','Fgf15','Crip1','Krt20','Dhcr24','Myo15b','Amn','Enpep','Anpep','Slc7a9','Ocm','Anxa2','Aoc1','Ceacam20','Arf6','Abcb1a','Xpnpep1','Vnn1','Cndp2','Nostrin','Slc13a1','Aspa','Maf','Myh14']
marker_genes['Goblet'] = ['Agr2', 'Fcgbp', 'Tff3', 'Clca1', 'Zg16', 'Tpsg1', 'Muc2', 'Galnt12', 'Atoh1', 'Rep15', 'S100a6', 'Pdia5', 'Klk1', 'Pla2g10', 'Spdef', 'Lrrc26', 'Ccl9', 'Bace2', 'Bcas1', 'Slc12a8', 'Smim14', 'Tspan13', 'Txndc5', 'Creb3l4', 'C1galt1c1', 'Creb3l1', 'Qsox1', 'Guca2a', 'Scin', 'Ern2', 'AW112010', 'Fkbp11', 'Capn9', 'Stard3nl', 'Slc50a1', 'Sdf2l1', 'Hgfa', 'Galnt7', 'Hpd', 'Ttc39a', 'Tmed3', 'Pdia6', 'Uap1', 'Gcnt3', 'Tnfaip8', 'Dnajc10', 'Ergic1', 'Tsta3', 'Kdelr3', 'Foxa3', 'Tpd52', 'Tmed9', 'Spink4', 'Nans', 'Cmtm7', 'Creld2', 'Tm9sf3', 'Wars', 'Smim6', 'Manf', 'Oit1', 'Tram1', 'Kdelr2', 'Xbp1', 'Serp1', 'Vimp', 'Guk1', 'Sh3bgrl3', 'Cmpk1', 'Tmsb10', 'Dap', 'Ostc', 'Ssr4', 'Sec61b', 'Pdia3', 'Gale', 'Klf4', 'Krtcap2', 'Arf4', 'Sep15', 'Ssr2', 'Ramp1', 'Calr', 'Ddost']
marker_genes['Paneth'] = ['Gm15284', 'AY761184', 'Defa17', 'Gm14851', 'Defa22', 'Defa-rs1', 'Defa3', 'Defa24', 'Defa26', 'Defa21', 'Lyz1', 'Gm15292', 'Mptx2', 'Ang4']
marker_genes['Enteroendocrine'] = ['Chgb', 'Gfra3', 'Cck', 'Vwa5b2', 'Neurod1', 'Fev', 'Aplp1', 'Scgn', 'Neurog3', 'Resp18', 'Trp53i11', 'Bex2', 'Rph3al', 'Scg5', 'Pcsk1', 'Isl1', 'Maged1', 'Fabp5', 'Celf3', 'Pcsk1n', 'Fam183b', 'Prnp', 'Tac1', 'Gpx3', 'Cplx2', 'Nkx2-2', 'Olfm1', 'Vim', 'Rimbp2', 'Anxa6', 'Scg3', 'Ngfrap1', 'Insm1', 'Gng4', 'Pax6', 'Cnot6l', 'Cacna2d1', 'Tox3', 'Slc39a2', 'Riiad1']
marker_genes['Tuft'] = ['Alox5ap', 'Lrmp', 'Hck', 'Avil', 'Rgs13', 'Ltc4s', 'Trpm5', 'Dclk1', 'Spib', 'Fyb', 'Ptpn6', 'Matk', 'Snrnp25', 'Sh2d7', 'Ly6g6f', 'Kctd12', '1810046K07Rik', 'Hpgds', 'Tuba1a', 'Pik3r5', 'Vav1', 'Tspan6', 'Skap2', 'Pygl', 'Ccdc109b', 'Ccdc28b', 'Plcg2', 'Ly6g6d', 'Alox5', 'Pou2f3', 'Gng13', 'Bmx', 'Ptpn18', 'Nebl', 'Limd2', 'Pea15a', 'Tmem176a', 'Smpx', 'Itpr2', 'Il13ra1', 'Siglecf', 'Ffar3', 'Rac2', 'Hmx2', 'Bpgm', 'Inpp5j', 'Ptgs1', 'Aldh2', 'Pik3cg', 'Cd24a', 'Ethe1', 'Inpp5d', 'Krt23', 'Gprc5c', 'Reep5', 'Csk', 'Bcl2l14', 'Tmem141', 'Coprs', 'Tmem176b', '1110007C09Rik', 'Ildr1', 'Galk1', 'Zfp428', 'Rgs2', 'Inpp5b', 'Gnai2', 'Pla2g4a', 'Acot7', 'Rbm38', 'Gga2', 'Myo1b', 'Adh1', 'Bub3', 'Sec14l1', 'Asah1', 'Ppp3ca', 'Agt', 'Gimap1', 'Krt18', 'Pim3', '2210016L21Rik', 'Tmem9', 'Lima1', 'Fam221a', 'Nt5c3', 'Atp2a3', 'Mlip', 'Vdac3', 'Ccdc23', 'Tmem45b', 'Cd47', 'Lect2', 'Pla2g16', 'Mocs2', 'Arpc5', 'Ndufaf3']

In [ ]:
cell_annotation = sc.tl.marker_gene_overlap(adata, marker_genes, key='rank_genes_leiden_r0.6')
cell_annotation

In [ ]:
cell_annotation_norm = sc.tl.marker_gene_overlap(adata, marker_genes, key='rank_genes_leiden_r0.6', normalize='reference')
sb.heatmap(cell_annotation_norm, cbar=False, annot=True)

In [ ]:
{
    'TA':0, # 1
    'EP (early)':1, # 0 
    'Stem':2, # 2
    'Goblet':3, #3
    'EP (stress)':4, # 6
    'Enterocyte':5, # 4
    'Paneth':6, # 5
    'Enteroendocrine':7,#8
    'Tuft':8}#7

In [ ]:
['EP (early)','TA','Stem','Goblet','Enterocyte','Paneth','EP (stress)','Tuft','Enteroendocrine']

In [ ]:
from matplotlib import colors
#Define a nice colour map for gene expression
colors2 = plt.cm.Reds(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,20))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)

In [ ]:
#Defa24 #Tff3
sc.pl.umap(adata, color='Defa24', use_raw=False, color_map=mymap)
sc.pl.umap(adata, color='Tff3', use_raw=False, color_map=mymap)

In [ ]:
# Check expression of enterocyte markers
#Collate all enterocyte markers and get the gene IDs in the data set
ids_entprox = np.in1d(adata.var_names, marker_genes['Enterocyte (Proximal)'])
ids_entdist = np.in1d(adata.var_names, marker_genes['Enterocyte (Distal)'])
ids_ent = np.logical_or(ids_entprox, ids_entdist)

#Calculate the mean expression of enterocyte markers
adata.obs['Enterocyte_marker_expr'] = adata.X[:,ids_ent].mean(1)

#Plot enterocyte expression
sc.pl.violin(adata, 'Enterocyte_marker_expr', groupby='leiden_r0.6')
sc.pl.umap(adata, color='Enterocyte_marker_expr', color_map=mymap)

In [ ]:
#Early enterocyte marker - Arg2
sc.pl.umap(adata, color='Arg2', use_raw=False, color_map=mymap)

sc.pl.violin(adata, groupby='leiden_r0.6', keys='Arg2', use_raw=False)

sc.pl.diffmap(adata, components=['6,9'], color='Arg2', use_raw=False, color_map=mymap)
sc.pl.diffmap(adata, components=['6,9'], color='leiden_r0.6')

In [ ]:
sc.pl.violin(adata, 'mt_frac', groupby='leiden_r0.6')
sc.pl.violin(adata, 'log_counts', groupby='leiden_r0.6')

In [ ]:

#Check individual stem markers
stem_genes = adata.var_names[np.in1d(adata.var_names, marker_genes['Stem'])]
sc.pl.umap(adata, color=stem_genes[:3], title=stem_genes[:3], color_map=mymap)
sc.pl.umap(adata, color=stem_genes[3:], title=stem_genes[3:], color_map=mymap)

In [ ]:
adata.obs['Stem_marker_expr'] = adata[:,stem_genes].X.mean(1)

sc.pl.violin(adata, 'Stem_marker_expr', groupby='leiden_r0.6')
sc.pl.umap(adata, color='Stem_marker_expr', color_map=mymap)

In [ ]:
#Categories to rename
adata.obs['leiden_r0.6'].cat.categories

In [ ]:
adata.rename_categories('leiden_r0.6',['EP (early)','TA','Stem','Goblet','Enterocyte','Paneth','EP (stress)','Tuft','Enteroendocrine'] )

In [ ]:
adata.obs['leiden_r0.6'].value_counts()

In [ ]:
plt.rcParams['figure.figsize']=(10,9)
sc.pl.umap(adata, color='leiden_r0.6', size=15, legend_loc='on data',palette=sc.pl.palettes.default_102)

### Subclustering

In [ ]:
#Subcluster enterocytes
sc.tl.louvain(adata, restrict_to=('leiden_r0.6', ['Enterocyte']), resolution=0.2, key_added='leiden_r0.6_entero_sub')

In [ ]:
#Show the new clustering
if 'leiden_r0.6_entero_sub_colors' in adata.uns:
    del adata.uns['leiden_r0.6_entero_sub_colors']

sc.pl.umap(adata, color='leiden_r0.6_entero_sub', palette=sc.pl.palettes.default_102)
sc.pl.umap(adata, color='region', palette=sc.pl.palettes.default_102)

In [ ]:
#Get the new marker genes
sc.tl.rank_genes_groups(adata, groupby='leiden_r0.6_entero_sub', key_added='rank_genes_r0.6_entero_sub')

In [ ]:
#Plot the new marker genes
sc.pl.rank_genes_groups(adata, key='rank_genes_r0.6_entero_sub', groups=['Enterocyte,0','Enterocyte,1','Enterocyte,2'], ncols=2, fontsize=12)

In [ ]:
entero_clusts = [clust for clust in adata.obs['leiden_r0.6_entero_sub'].cat.categories if clust.startswith('Enterocyte')]

for clust in entero_clusts:
    sc.pl.rank_genes_groups_violin(adata, use_raw=True, key='rank_genes_r0.6_entero_sub', groups=[clust], gene_names=adata.uns['rank_genes_r0.6_entero_sub']['names'][clust][90:100])

In [ ]:
#Subset marker gene dictionary to only check for enterocyte markers
marker_genes_entero = {k: marker_genes[k] for k in marker_genes if k.startswith('Enterocyte')}

In [ ]:
#Find marker overlap
sc.tl.marker_gene_overlap(adata, marker_genes_entero, key='rank_genes_r0.6_entero_sub', normalize='reference')

In [ ]:
#Check enterocyte marker expression
sc.pl.violin(adata[adata.obs['leiden_r0.6']=='Enterocyte'], groupby='leiden_r0.6_entero_sub', keys='Enterocyte_marker_expr')

In [ ]:
#Visualize some enterocyte markers
entero_genes = ['Alpi', 'Apoa1', 'Apoa4', 'Fabp1', 'Arg2']
sc.pl.umap(adata, color=entero_genes[:3], title=entero_genes[:3], color_map=mymap)
sc.pl.umap(adata, color=entero_genes[3:], title=entero_genes[3:], color_map=mymap)

In [ ]:
sc.pl.diffmap(adata, color='leiden_r0.6_entero_sub', components='3,7')

In [ ]:
tmp = adata.obs['leiden_r0.6_entero_sub'].cat.categories

tmp = ['Enterocyte imm. (Distal)' if item == 'Enterocyte,0' else item for item in tmp]
tmp = ['Enterocyte imm. (Proximal)' if item == 'Enterocyte,2' else item for item in tmp]
tmp = ['Enterocyte mature' if item == 'Enterocyte,1' else item for item in tmp]


adata.rename_categories('leiden_r0.6_entero_sub', tmp)

In [ ]:
#Subcluster mature enterocytes
sc.tl.louvain(adata, restrict_to=('leiden_r0.6_entero_sub', ['Enterocyte mature']), resolution=0.25, key_added='leiden_r0.6_entero_mat_sub')

In [ ]:
#Show the new clustering
if 'leiden_r0.6_entero_mat_sub_colors' in adata.uns:
    del adata.uns['leiden_r0.6_entero_mat_sub_colors']

sc.pl.umap(adata, color='leiden_r0.6_entero_mat_sub', palette=sc.pl.palettes.default_102)

In [ ]:
#Get the new marker genes
sc.tl.rank_genes_groups(adata, groupby='leiden_r0.6_entero_mat_sub', key_added='rank_genes_r0.6_entero_mat_sub')

In [ ]:
#Plot the new marker genes
sc.pl.rank_genes_groups(adata, key='rank_genes_r0.6_entero_mat_sub',
                        groups=['Enterocyte mature,0','Enterocyte mature,1'], fontsize=12)

In [ ]:
pd.DataFrame(adata.uns['rank_genes_r0.6_entero_mat_sub']['names'])['Enteroendocrine'].head(15).to_list()

In [ ]:
entero_mat_clusts = [clust for clust in adata.obs['leiden_r0.6_entero_mat_sub'].cat.categories \
                       if clust.startswith('Enterocyte mature')]

for clust in entero_mat_clusts:
    sc.pl.rank_genes_groups_violin(
        adata, use_raw=True, key='rank_genes_r0.6_entero_mat_sub', groups=[clust], 
        gene_names=adata.uns['rank_genes_r0.6_entero_mat_sub']['names'][clust][90:100])

In [ ]:
#Find marker overlap
sc.tl.marker_gene_overlap(adata, marker_genes_entero, key='rank_genes_r0.6_entero_mat_sub', normalize='reference')

In [ ]:
tmp = adata.obs['leiden_r0.6_entero_mat_sub'].cat.categories

tmp = ['Enterocyte mat. (Distal)' if item == 'Enterocyte mature,0' else item for item in tmp]
tmp = ['Enterocyte mat. (Proximal)' if item == 'Enterocyte mature,1' else item for item in tmp]

adata.rename_categories('leiden_r0.6_entero_mat_sub', tmp)

In [ ]:
adata.obs['leiden_final'] = adata.obs['leiden_r0.6_entero_mat_sub']

In [ ]:
sc.pl.umap(adata, color='leiden_final', palette=sc.pl.palettes.default_102, legend_loc='on data',legend_fontsize=8)

In [ ]:
adata.obs['leiden_final'].value_counts()

### Compositional analysis
We can visualize shifts in cellular densities between conditions

In [ ]:
#Define a variable that stores proximal and distal labels
adata.obs['prox_dist'] = ['Distal' if reg=='Il' else 'Proximal' for reg in adata.obs['region']]

In [ ]:
sc.tl.embedding_density(adata, basis='umap', groupby='prox_dist')

In [ ]:
adata.obs['prox_dist'].value_counts()
sc.pl.embedding_density(adata, basis='umap', key='umap_density_prox_dist', group='Proximal')
sc.pl.embedding_density(adata, basis='umap', key='umap_density_prox_dist', group='Distal')

### Trajectory inference and pseudotime analysis

In [ ]:
adata.obs['leiden_final'].value_counts()

In [ ]:
#Subsetting to relevant clusters
clusters_to_include = [g for g in adata.obs['leiden_final'].cat.categories \
                         if (g.startswith('Enterocyte') or \
                             g.startswith('TA') or \
                             g.startswith('Stem') or \
                             g.startswith('EP'))]
adata_ent = adata[np.isin(adata.obs['leiden_final'], clusters_to_include),:].copy()

#Subset to highly variable genes
sc.pp.highly_variable_genes(adata_ent, flavor='cell_ranger', n_top_genes=4000, subset=True)

In [ ]:
#Recalculating PCA for subset
sc.pp.pca(adata_ent, svd_solver='arpack')
sc.pl.pca(adata_ent)
sc.pl.pca_variance_ratio(adata_ent)

In [ ]:
adata_ent.obsm['X_pca'] = adata_ent.obsm['X_pca'][:,0:7]

### Slingshot

In [ ]:
%%R -i adata_ent

#Plot 1
colour_map = brewer.pal(20,'Set1')
par(xpd=TRUE)
par(mar=c(4.5,5.5,2,7))
plot(reducedDims(adata_ent)$PCA[,1], reducedDims(adata_ent)$PCA[,2], col=colour_map[colData(adata_ent)$leiden_final], bty='L', xlab='PC1', ylab='PC2')
legend(x=12, y=12, legend=unique(colData(adata_ent)$leiden_final), fill=colour_map[as.integer(unique(colData(adata_ent)$leiden_final))])

print("1:")
adata_ent_start <- slingshot(adata_ent, clusterLabels = 'leiden_final', reducedDim = 'PCA', start.clus='Stem')
print(SlingshotDataSet(adata_ent_start))

print("")
print("2:")
adata_ent_startend <- slingshot(adata_ent, clusterLabels = 'leiden_final', reducedDim = 'PCA', start.clus='Stem', end.clus=c('Enterocyte mat. (Proximal)', 'Enterocyte mat. (Distal)'))
print(SlingshotDataSet(adata_ent_startend))

print("")
print("3:")
adata_ent_simple_startend <- slingshot(adata_ent, clusterLabels = 'leiden_r0.6', reducedDim = 'PCA', start.clus='Stem', end.clus='Enterocyte')
print(SlingshotDataSet(adata_ent_simple_startend))

In [ ]:
%%R
options(repr.plot.width=10, repr.plot.height=12)
plot(reducedDims(adata_ent)$PCA[,1], reducedDims(adata_ent)$PCA[,2], col=colour_map[colData(adata_ent)$leiden_final], bty='L', xlab='PC1', ylab='PC2')
legend(x=12, y=12, legend=unique(colData(adata_ent)$leiden_final), fill=colour_map[as.integer(unique(colData(adata_ent)$leiden_final))])


In [ ]:
%%R

#Plot of lineage 1
colors <- colorRampPalette(brewer.pal(11,'Spectral')[-6])(100)
plot(reducedDims(adata_ent_startend)$PCA[,c(1,2)], col = colors[cut(adata_ent_startend$slingPseudotime_1,breaks=100)], pch=16, asp = 1, xlab='PC1', ylab='PC2')
lines(slingCurves(adata_ent_startend)$curve1, lwd=2)

#Plot of lineage 2
colors <- colorRampPalette(brewer.pal(11,'Spectral')[-6])(100)
plot(reducedDims(adata_ent_startend)$PCA[,c(1,2)], col = colors[cut(adata_ent_startend$slingPseudotime_2,breaks=100)], pch=16, asp = 1, xlab='PC1', ylab='PC2')
lines(slingCurves(adata_ent_startend)$curve2, lwd=2)

#Plot of lineages with clusters visualized
par(xpd=TRUE)
plot(reducedDims(adata_ent_startend)$PCA[,c(1,2)], col = brewer.pal(11,'Set1')[adata_ent$leiden_final], pch=16, asp = 1, bty='L', xlab='PC1', ylab='PC2')
lines(SlingshotDataSet(adata_ent_startend), lwd=2, type='lineages')
legend(x=10, y=20, legend=unique(colData(adata_ent)$leiden_final), fill=brewer.pal(11,'Set1')[as.integer(unique(colData(adata_ent)$leiden_final))])

#Plot of simpler clustering
plot(reducedDims(adata_ent_simple_startend)$PCA[,c(1,2)], col = colors[cut(adata_ent_simple_startend$slingPseudotime_1,breaks=100)], pch=16, asp = 1, xlab='PC1', ylab='PC2')
lines(SlingshotDataSet(adata_ent_simple_startend), lwd=2)

### Diffusion Pseudotime (DPT)

In [ ]:
sc.pp.neighbors(adata_ent)
sc.tl.diffmap(adata_ent)

In [ ]:
sc.pl.diffmap(adata_ent, components='1,2', color='leiden_final')
sc.pl.diffmap(adata_ent, components='1,3', color='leiden_final')

In [ ]:
#Find the stem cell with the highest DC3 value to act as root for the diffusion pseudotime and compute DPT
stem_mask = np.isin(adata_ent.obs['leiden_final'], 'Stem')
max_stem_id = np.argmin(adata_ent.obsm['X_diffmap'][stem_mask,3])
root_id = np.arange(len(stem_mask))[stem_mask][max_stem_id]
adata_ent.uns['iroot'] = root_id

#Compute dpt
sc.tl.dpt(adata_ent)

In [ ]:
#Visualize pseudotime over differentiation
sc.pl.diffmap(adata_ent, components='1,3', color='dpt_pseudotime')

## Gene expression dynamics

In [ ]:
%%R

#Set the pseudotime variable
t <- adata_ent_simple_startend$slingPseudotime_1

#Extract the gene expression matrix
Y <- assay(adata_ent_simple_startend)

# fit a GAM with a loess term for pseudotime
#Note: This takes a while
gam.pval <- apply(Y,1,function(z){
  d <- data.frame(z=z, t=t)
  tmp <- gam(z ~ lo(t), data=d)
  p <- summary(tmp)[4][[1]][1,5]
  p
})

In [ ]:
%%R -w 600 -h 1200

#Select the top 100 most significant genes that change over pseudotime
topgenes <- names(sort(gam.pval, decreasing = FALSE))[1:100]
heatdata <- assay(adata_ent_simple_startend)[rownames(assay(adata_ent_simple_startend)) %in% topgenes, 
                        order(t, na.last = NA)]

#Scale the data per gene for visualization
heatdata <- t(scale(t(heatdata)))

#Trimm z-score scale
heatdata[heatdata > 3] = 3
heatdata[heatdata < -3] = -3

#Get cluster assignment
heatclus <- adata_ent_simple_startend$leiden_r0.6[order(t, na.last = NA)]

#Set up a clusterExperiment data structure for heatmap visualization
ce <- ClusterExperiment(heatdata, heatclus, transformation = function(x){x})

#Plot the heatmap
plotHeatmap(ce, clusterSamplesData = "orderSamplesValue", visualizeData = 'transformed', fontsize=15)


## Partition-based graph abstraction

In [ ]:
sc.tl.paga(adata, groups='leiden_r0.6')

In [ ]:
adata.uns.get('paga')

In [ ]:
sc.pl.paga(adata, color='leiden_r0.6',cmap=sc.pl.palettes.default_102)
sc.pl.umap(adata, color='leiden_r0.6', palette=sc.pl.palettes.default_102)

In [ ]:
sc.tl.draw_graph(adata, init_pos='paga')

In [ ]:
sc.pl.draw_graph(adata, color='leiden_r0.6',palette=sc.pl.palettes.default_102)

In [ ]:
plt.rcParams['figure.figsize']=(8,8)
sc.pl.paga_compare(adata, basis='umap')

In [ ]:
fig1, ax1 = plt.subplots()
sc.pl.umap(adata, size=50, ax=ax1,show=False)
sc.pl.paga(adata, pos=adata.uns['paga']['pos'],show=False,  node_size_scale=2, node_size_power=1, ax=ax1, text_kwds={'alpha':0})

## Gene-level analysis

### Gene set analysis

In [ ]:
gp = GProfiler(return_dataframe=True, user_agent='g:GOSt')

In [ ]:
endo_genes = ['Fam183b',
 'Gfra3',
 'Aplp1',
 'Chgb',
 'Tm4sf4',
 'Cdkn1a',
 'Gch1',
 'Marcksl1',
 'Ddc',
 'Cpe',
 'Cystm1',
 'Fxyd3',
 'Ngfrap1',
 'Fev',
 'Cryba2']

In [ ]:
paneth_enrichment = gp.profile(organism='mmusculus', sources=['GO:BP'], user_threshold=0.05,
                               significance_threshold_method='fdr', 
                               background=adata.var_names.tolist(), 
                               query=endo_genes)

In [ ]:
paneth_enrich_results = paneth_enrichment.set_index('native').sort_values('p_value').iloc[:,[2,5,7,10,1]]

In [ ]:
pd.set_option("display.max_colwidth", 800)
paneth_enrich_results.iloc[:50,:]

In [ ]:
## codes copied from https://github.com/theislab/single-cell-tutorial

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from matplotlib import colors
from matplotlib import rcParams


def scale_data_5_75(data):
    mind = np.min(data)
    maxd = np.max(data)
    
    if maxd == mind:
        maxd=maxd+1
        mind=mind-1
        
    drange = maxd - mind
    return ((((data - mind)/drange*0.70)+0.05)*100)


def plot_enrich(data, n_terms=20, save=False):
    # Test data input
    if not isinstance(data, pd.DataFrame):
        raise ValueError('Please input a Pandas Dataframe output by gprofiler.')
        
    if not np.all([term in data.columns for term in ['p_value', 'name', 'intersection_size']]):
        raise TypeError('The data frame {} does not contain enrichment results from gprofiler.'.format(data))
    
    data_to_plot = data.iloc[:n_terms,:].copy()
    data_to_plot['go.id'] = data_to_plot.index

    min_pval = data_to_plot['p_value'].min()
    max_pval = data_to_plot['p_value'].max()
    
    # Scale intersection_size to be between 5 and 75 for plotting
    #Note: this is done as calibration was done for values between 5 and 75
    data_to_plot['scaled.overlap'] = scale_data_5_75(data_to_plot['intersection_size'])
    
    norm = colors.LogNorm(min_pval, max_pval)
    sm = plt.cm.ScalarMappable(cmap="cool", norm=norm)
    sm.set_array([])

    rcParams.update({'font.size': 14, 'font.weight': 'bold'})

    sb.set(style="whitegrid")

    path = plt.scatter(x='recall', y="name", c='p_value', cmap='cool', 
                       norm=colors.LogNorm(min_pval, max_pval), 
                       data=data_to_plot, linewidth=1, edgecolor="grey", 
                       s=[(i+10)**1.5 for i in data_to_plot['scaled.overlap']])
    ax = plt.gca()
    ax.invert_yaxis()

    ax.set_ylabel('')
    ax.set_xlabel('Gene ratio', fontsize=14, fontweight='bold')
    ax.xaxis.grid(False)
    ax.yaxis.grid(True)

    # Shrink current axis by 20%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

    # Get tick marks for this plot
    #Note: 6 ticks maximum
    min_tick = np.floor(np.log10(min_pval)).astype(int)
    max_tick = np.ceil(np.log10(max_pval)).astype(int)
    tick_step = np.ceil((max_tick - min_tick)/6).astype(int)
    
    # Ensure no 0 values
    if tick_step == 0:
        tick_step = 1
        min_tick = max_tick-1
    
    ticks_vals = [10**i for i in range(max_tick, min_tick-1, -tick_step)]
    ticks_labs = ['$10^{'+str(i)+'}$' for i in range(max_tick, min_tick-1, -tick_step)]

    #Colorbar
    fig = plt.gcf()
    cbaxes = fig.add_axes([0.8, 0.15, 0.03, 0.4])
    cbar = ax.figure.colorbar(sm, ticks=ticks_vals, shrink=0.5, anchor=(0,0.1), cax=cbaxes)
    cbar.ax.set_yticklabels(ticks_labs)
    cbar.set_label("Adjusted p-value", fontsize=14, fontweight='bold')

    #Size legend
    min_olap = data_to_plot['intersection_size'].min()
    max_olap = data_to_plot['intersection_size'].max()
    olap_range = max_olap - min_olap
    
    #Note: approximate scaled 5, 25, 50, 75 values are calculated
    #      and then rounded to nearest number divisible by 5
    size_leg_vals = [np.round(i/5)*5 for i in 
                          [min_olap, min_olap+(20/70)*olap_range, min_olap+(45/70)*olap_range, max_olap]]
    size_leg_scaled_vals = scale_data_5_75(size_leg_vals)

    
    l1 = plt.scatter([],[], s=(size_leg_scaled_vals[0]+10)**1.5, edgecolors='none', color='black')
    l2 = plt.scatter([],[], s=(size_leg_scaled_vals[1]+10)**1.5, edgecolors='none', color='black')
    l3 = plt.scatter([],[], s=(size_leg_scaled_vals[2]+10)**1.5, edgecolors='none', color='black')
    l4 = plt.scatter([],[], s=(size_leg_scaled_vals[3]+10)**1.5, edgecolors='none', color='black')

    labels = [str(int(i)) for i in size_leg_vals]

    leg = plt.legend([l1, l2, l3, l4], labels, ncol=1, frameon=False, fontsize=12,
                     handlelength=1, loc = 'center left', borderpad = 1, labelspacing = 1.4,
                     handletextpad=2, title='Gene overlap', scatterpoints = 1,  bbox_to_anchor=(-2, 1.5), 
                     facecolor='black')

    if save:
        plt.savefig(save, dpi=300, format='pdf')

    plt.show()

In [ ]:
plot_enrich(paneth_enrich_results)